In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/H6751 Text & Web Mining Project
#%cd /content/drive/My Drive/H6751 Kaggle

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/H6751 Text & Web Mining Project


In [2]:
!pip install inflect 

import re
import os
import time
import math
import numpy as np
import pandas as pd
import nltk
import inflect
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import codecs
from nltk.stem import PorterStemmer
from sklearn.model_selection import GridSearchCV
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input,  Dense, Flatten, Concatenate, MaxPooling2D, LeakyReLU, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from string import digits

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
#===================================
# utilities
#===================================
def pd_remove_url(data):
  no_row = len(data.index)
  for i, sen in enumerate(data, 0):
    data.iloc[i] = re.sub(r'https?:\/\/.*[\r\n]*', '', sen)
  return data

def pd_remove_stopwords(data):
    stop_words = stopwords.words('english')
    for i, sen in enumerate(data, 0):
      new_sen = []
      for word in sen.split():
        if word not in stop_words:
            new_sen.append(word)
      data.iloc[i] = ' '.join(new_sen)
    return data

def pd_remove_punctuation(data):
    for i, sen in enumerate(data, 0):
      new_sen = []
      for word in sen.split():
        new_word = re.sub(r'[^\w\s]', ' ', word)
        if new_word != '':
            new_sen.append(new_word)
      data.iloc[i] = ' '.join(new_sen)
    return data

def pd_stem_text(data):
    stemmer= PorterStemmer()
    for i, sen in enumerate(data, 0):
      new_sen = []
      for word in sen.split():
        new_sen.append(stemmer.stem(word))
      data.iloc[i] = ' '.join(new_sen)
    return data

def normaliz(v):
  norm_v = v/np.linalg.norm(v)
  return norm_v

def get_digit(s):
    return sum(c.isdigit() for c in s)
    
def feat_eng(data):
  char_count = normaliz(data.apply(len).values.reshape(-1,1))
  word_count = normaliz(data.apply(lambda x: len(x.split())).values.reshape(-1,1))
  word_density = normaliz(char_count / (word_count+1))
  punctuation_count = normaliz(data.apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))).values.reshape(-1,1))
  digit_count = normaliz(data.apply(get_digit).values.reshape(-1,1))
  return char_count, word_count, word_density, punctuation_count, digit_count

def print_result(model,model_name,x_dev,y_dev):
  y_pred = (model.predict(x_dev) > 0.5).astype("int32")
  y_true = y_dev
  print(model_name,' accurary: ',accuracy_score(y_true, y_pred))
  print(model_name,' f1-score: ',f1_score(y_true, y_pred))
  print('------------------------------------------------------------')

def averageLen(lst):
    lengths = [len(i) for i in lst]
    return 0 if len(lengths) == 0 else (float(sum(lengths)) / len(lengths)) 


def checkmorethan(thre, lst):
  count = 0
  for l in lst:
    if len(l) > thre:
      count += 1
  return count

np.random.seed(2568)


In [4]:
# data loading
test_csv = pd.read_csv("data/test.csv")
train_csv = pd.read_csv("data/train.csv")
target_column = 'target'
text_column = 'text'

X = train_csv.text
Y = train_csv.target
test_text = test_csv.text
train_text, dev_text, y_train, y_dev = train_test_split(X, Y, test_size=0.5, random_state=2568)

#========================================
# parameters
#========================================
remove_url = False
remove_punctuation = True
remove_stopwords = False
stem_text = False
test_size = 0.2

#========================================
# data preprocessing
#========================================
if remove_url:
  pd_remove_url(X)
  pd_remove_url(test_text)

if remove_punctuation:
  pd_remove_punctuation(X)
  pd_remove_punctuation(test_text)

if remove_stopwords:
  pd_remove_stopwords(X)
  pd_remove_stopwords(test_text)

if stem_text:
  pd_stem_text(X)
  pd_stem_text(test_text)

train_text, dev_text, y_train, y_dev = train_test_split(X, Y, test_size=0.5, random_state=2568)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# tokenization
train = {} ; dev = {} 

# word tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
x_train = tokenizer.texts_to_sequences(train_text)
x_dev = tokenizer.texts_to_sequences(dev_text)
x_test = tokenizer.texts_to_sequences(test_text)
word_to_id = tokenizer.word_index

In [0]:
#===============================
# examine data
#===============================
print('total', len(x_train))
print('more than 256word:',checkmorethan(256,x_train ), ', ratio:' ,checkmorethan(256,x_train )/len(x_train) )
print('more than 1000char:',checkmorethan(1000,x_train_char ), ', ratio:' ,checkmorethan(1000,x_train_char )/len(x_train_char) )
print('average char len:',averageLen(x_train_char))
print('average word len:',averageLen(x_train))
print('max char len:',len(max(x_train_char, key=len)))
print('max word len:',len(max(x_train, key=len)))

total 3806
more than 256word: 0 , ratio: 0.0
more than 1000char: 0 , ratio: 0.0
average char len: 100.73673147661587
average word len: 17.076983709931685
max char len: 151
max word len: 32


In [0]:
#=========================================
# GloVe Loading
#=========================================
glove_loc = '/content/drive/My Drive/H6751 Text & Web Mining Project/data/glove.6B.100d.txt'
use_glove = True
word_dim = 100

if use_glove:
  # load glove embedding
  all_word_embeds = {}
  for i, line in enumerate(codecs.open(glove_loc, 'r', 'utf-8')):
      s = line.strip().split()
      if len(s) == word_dim + 1:
          all_word_embeds[s[0]] = np.array([float(i) for i in s[1:]])

  #Intializing Word Embedding Matrix
  word_embeds = np.random.uniform(-np.sqrt(0.06), np.sqrt(0.06), (len(word_to_id)+1, word_dim))

  for w in word_to_id:
      if w in all_word_embeds:
          word_embeds[word_to_id[w]] = all_word_embeds[w]
      elif w.lower() in all_word_embeds:
          word_embeds[word_to_id[w]] = all_word_embeds[w.lower()]

  print('Loaded %i pretrained embeddings.' % len(all_word_embeds))


Loaded 400000 pretrained embeddings.


In [0]:
#=========================================
# CNN-1D
#=========================================
# Embedding
maxlen = 32
embedding_size = 100

# Convolution
kernel_size = 3
filters = 100
pool_size = 4

# LSTM
lstm_output_size = 100

# Training
batch_size = 32
epochs = 25
dropout = 0.8

# Glove setting
use_glove = True
train_glove = False

# save best model for eval
best_model_name = 'model.best_cnn'
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
mcp_save = ModelCheckpoint(best_model_name, save_best_only=True, monitor='val_accuracy', mode='max')

print(len(x_train), 'train sequences')
print(len(x_dev), 'dev sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_dev = sequence.pad_sequences(x_dev, maxlen=maxlen, padding='post')

print('x_train shape:', x_train.shape)
print('x_dev shape:', x_dev.shape)
print('Build model...')

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
if use_glove:
  model.add(Embedding(len(word_embeds), embedding_size, weights=[word_embeds], input_length=maxlen, trainable=train_glove))
else:
  model.add(Embedding(len(word_to_id)+1, embedding_size, input_length=maxlen))

model.add(Dropout(dropout))

model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(100))
model.add(Dropout(dropout))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[earlyStopping, mcp_save],
          validation_data=(x_dev, y_dev))

model = load_model(best_model_name)
print_result(model,best_model_name,x_dev,y_dev)

3806 train sequences
3807 dev sequences
Pad sequences (samples x time)
x_train shape: (3806, 32)
x_dev shape: (3807, 32)
Build model...
Epoch 1/25
119/119 [==============================] - 2s 17ms/step - loss: 0.7861 - accuracy: 0.5554 - val_loss: 0.6739 - val_accuracy: 0.5808
Epoch 2/25
119/119 [==============================] - 2s 17ms/step - loss: 0.6791 - accuracy: 0.5704 - val_loss: 0.6594 - val_accuracy: 0.5879
Epoch 3/25
119/119 [==============================] - 2s 16ms/step - loss: 0.6623 - accuracy: 0.5956 - val_loss: 0.6405 - val_accuracy: 0.7373
Epoch 4/25
119/119 [==============================] - 1s 7ms/step - loss: 0.6519 - accuracy: 0.6174 - val_loss: 0.6173 - val_accuracy: 0.7147
Epoch 5/25
119/119 [==============================] - 2s 17ms/step - loss: 0.6364 - accuracy: 0.6466 - val_loss: 0.5822 - val_accuracy: 0.7746
Epoch 6/25
119/119 [==============================] - 2s 17ms/step - loss: 0.6133 - accuracy: 0.6713 - val_loss: 0.5547 - val_accuracy: 0.7799
Epoch 7

In [0]:
#=========================================
# Bi-LSTM
#=========================================
# Embedding
maxlen = 32
embedding_size = 100

# Convolution
kernel_size = 3
filters = 100
pool_size = 4

# LSTM
lstm_output_size = 100

# Training
batch_size = 32
epochs = 25
dropout = 0.8

# Glove setting
use_glove = True
train_glove = False

# save best model for eval
best_model_name = 'model.best_bilstm'
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
mcp_save = ModelCheckpoint(best_model_name, save_best_only=True, monitor='val_accuracy', mode='max')

print(len(x_train), 'train sequences')
print(len(x_dev), 'dev sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_dev = sequence.pad_sequences(x_dev, maxlen=maxlen, padding='post')

print('x_train shape:', x_train.shape)
print('x_dev shape:', x_dev.shape)
print('Build model...')


model = Sequential()

if use_glove:
  model.add(Embedding(len(word_embeds), embedding_size, weights=[word_embeds], input_length=maxlen, trainable=train_glove))
else:
  model.add(Embedding(len(word_to_id)+1, embedding_size, input_length=maxlen))

model.add(Dropout(dropout))

# Bi-LSTM layer
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(dropout))
model.add(Activation('relu'))

# We add a vanilla hidden layer:
model.add(Dense(100))
model.add(Dropout(dropout))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[earlyStopping, mcp_save],
          validation_data=(x_dev, y_dev))

model = load_model(best_model_name)
print_result(model,best_model_name,x_dev,y_dev)

3806 train sequences
3807 dev sequences
Pad sequences (samples x time)
x_train shape: (3806, 32)
x_dev shape: (3807, 32)
Build model...
Epoch 1/25
119/119 [==============================] - 20s 165ms/step - loss: 0.7044 - accuracy: 0.5457 - val_loss: 0.6763 - val_accuracy: 0.5692
Epoch 2/25
119/119 [==============================] - 3s 23ms/step - loss: 0.6894 - accuracy: 0.5549 - val_loss: 0.6712 - val_accuracy: 0.5661
Epoch 3/25
119/119 [==============================] - 19s 161ms/step - loss: 0.6763 - accuracy: 0.5723 - val_loss: 0.6437 - val_accuracy: 0.6341
Epoch 4/25
119/119 [==============================] - 19s 161ms/step - loss: 0.6417 - accuracy: 0.6243 - val_loss: 0.5724 - val_accuracy: 0.7147
Epoch 5/25
119/119 [==============================] - 18s 153ms/step - loss: 0.6320 - accuracy: 0.6456 - val_loss: 0.5506 - val_accuracy: 0.7258
Epoch 6/25
119/119 [==============================] - 20s 165ms/step - loss: 0.6012 - accuracy: 0.7018 - val_loss: 0.5301 - val_accuracy: 0.7

In [0]:
#=========================================
# CNN + Bi-LSTM
#=========================================
# Embedding
maxlen = 32
embedding_size = 100

# Convolution
kernel_size = 3
filters = 100
pool_size = 4

# LSTM
lstm_output_size = 100

# Glove setting
use_glove = True
train_glove = False

# Training
batch_size = 32
epochs = 25
dropout = 0.8

# save best model for eval
best_model_name = 'model.best_fusion'
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
mcp_save = ModelCheckpoint(best_model_name, save_best_only=True, monitor='val_accuracy', mode='max')

print(len(x_train), 'train sequences')
print(len(x_dev), 'dev sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_dev = sequence.pad_sequences(x_dev, maxlen=maxlen, padding='post')

print('x_train shape:', x_train.shape)
print('x_dev shape:', x_dev.shape)
print('Build model...')

model = Sequential()

if use_glove:
  model.add(Embedding(len(word_embeds), embedding_size, weights=[word_embeds], input_length=maxlen, trainable=train_glove))
else:
  model.add(Embedding(len(word_to_id)+1, embedding_size, input_length=maxlen))

model.add(Dropout(dropout))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))

model.add(Bidirectional(LSTM(lstm_output_size)))
model.add(Dropout(dropout))
model.add(Activation('relu'))

# We add a vanilla hidden layer:
model.add(Dense(100))
model.add(Dropout(dropout))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[earlyStopping, mcp_save],
          validation_data=(x_dev, y_dev))

model = load_model(best_model_name)
print_result(model,best_model_name,x_dev,y_dev)

3806 train sequences
3807 dev sequences
Pad sequences (samples x time)
x_train shape: (3806, 32)
x_dev shape: (3807, 32)
Build model...
Epoch 1/25
119/119 [==============================] - 19s 162ms/step - loss: 0.7064 - accuracy: 0.5347 - val_loss: 0.6857 - val_accuracy: 0.5647
Epoch 2/25
119/119 [==============================] - 3s 26ms/step - loss: 0.6820 - accuracy: 0.5591 - val_loss: 0.6511 - val_accuracy: 0.5647
Epoch 3/25
119/119 [==============================] - 20s 166ms/step - loss: 0.6534 - accuracy: 0.6054 - val_loss: 0.5720 - val_accuracy: 0.7583
Epoch 4/25
119/119 [==============================] - 20s 166ms/step - loss: 0.6158 - accuracy: 0.6718 - val_loss: 0.5263 - val_accuracy: 0.7623
Epoch 5/25
119/119 [==============================] - 19s 158ms/step - loss: 0.6001 - accuracy: 0.6984 - val_loss: 0.4926 - val_accuracy: 0.7770
Epoch 6/25
119/119 [==============================] - 4s 38ms/step - loss: 0.5827 - accuracy: 0.7178 - val_loss: 0.5327 - val_accuracy: 0.733